#Import

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Widgets

In [0]:
dbutils.widgets.removeAll()
dbutils.widgets.text("catalog","catalog_ecommerce")
dbutils.widgets.text("schema_source","silver")
dbutils.widgets.text("shema_sink","gold")

In [0]:
catalog = dbutils.widgets.get("catalog")
schema_source = dbutils.widgets.get("schema_source")
schema_sink = dbutils.widgets.get("shema_sink")

#Metricas

In [0]:
df_sales_monthly = spark.sql(f"""
    SELECT 
        date_trunc('month', o.order_purchase_timestamp) as month,
        year(o.order_purchase_timestamp) as year,
        month(o.order_purchase_timestamp) as month_number,
            
        -- conteo de ordenes
        count(DISTINCT o.order_id) as total_orders,
        count(DISTINCT o.customer_id) as customers,
            
        -- Metricas de revenue
        round(SUM(oi.Total_pay), 2) as total_revenue,
        round(AVG(oi.Total_pay), 2) as avg_order_value,
        round(SUM(oi.freight_value), 2) as total_freight_value,
            
        -- Metricas de items
        count(oi.order_item_id) as total_items_sold,
        round(AVG(oi.price), 2) as avg_item_price,
            
        -- Metricas de entrega
        round(AVG(o.delivery_days), 1) as avg_delivery_days,
        sum(CASE WHEN o.Is_late THEN 1 ELSE 0 END) as late_deliveries,
            
        -- Estado de ordenes
        sum(CASE WHEN o.order_status = 'delivered' THEN 1 ELSE 0 END) as delivered_orders,
        sum(CASE WHEN o.order_status = 'canceled' THEN 1 ELSE 0 END) as canceled_orders
    
    FROM {catalog}.{schema_source}.orders o
    INNER JOIN {catalog}.{schema_source}.orders_items oi 
        ON o.order_id = oi.order_id
    GROUP BY 
        date_trunc('month', o.order_purchase_timestamp),
        year(o.order_purchase_timestamp),
        month(o.order_purchase_timestamp)
    ORDER BY month DESC
""")

In [0]:
df_sales_monthly.write.mode("overwrite").saveAsTable(f"{catalog}.{schema_sink}.Ventas_mensuales")